In [1]:
import pandas as pd
import numpy as np

$$
\begin{aligned}
&\frac{\partial L}{\partial p_u}=0 \Rightarrow p_u=\left(Q^TC_u Q+\lambda I\right)^{-1} Q^T C_u \phi_u \\
&\frac{\partial L}{\partial q_i}=0 \Rightarrow q_i=\left(P^TC_i P+\lambda I\right)^{-1} P^T C_i \phi_i
\end{aligned}
$$

$$
\begin{aligned}
&\frac{\partial L}{\partial p_u}=0 \Rightarrow p_u=\left(Q^T Q+Q^T\left(C_u-I\right) Q+\lambda I\right)^{-1} Q^T C_u \phi_u \\
&\frac{\partial L}{\partial q_i}=0 \Rightarrow q_i=\left(P^T P+P^T\left(C_i-I\right) P+\lambda I\right)^{-1} P^T C_i \phi_i
\end{aligned}
$$

$$ L\left(p_u, q_i, r_{u i}\right)=\sum_{u, i \in R} c_{u i} \cdot\left(\phi_{u i}-\langle p_u, q_i \rangle\right)^2=\sum_{u, i \in R}\left(1+\alpha r_{u i}\right)\left(\phi_{u i}-\langle p_u, q_i \rangle\right)^2 $$

In [11]:
class ImplicitALS:
    def __init__(self, R, k=3, a=40, lambd=10):
        self.lambd = lambd
        self.k = k
        self.a = a
        self.m, self.n = R.shape
        self.P = np.random.rand(self.m, self.k)
        self.Q = np.random.rand(self.n, self.k)
        self.phi = np.where(R > 0, 1, 0)
        self.C = 1 + a * R
        self.loss = []

    def train(self, n_step=10):
        l2_reg = self.lambd * np.identity(self.k)
        for _ in range(n_step):
            
            # fix P, update Q
            PtP = self.P.T.dot(self.P)
            
            for i in range(self.n):
                Ci = np.diag(self.C[:, i])
                Wi = PtP + self.P.T.dot(Ci - np.identity(self.m)).dot(self.P) + l2_reg
                self.Q[i] = np.linalg.inv(Wi).dot(self.P.T).dot(Ci).dot(self.phi[:, i])
                
            # fix Q, update P
            QtQ = self.Q.T.dot(self.Q)
            for u in range(self.m):
                Cu = np.diag(self.C[u, :])
                Wu = QtQ + self.Q.T.dot(Cu - np.identity(self.n)).dot(self.Q) + l2_reg
                self.P[u] = np.linalg.inv(Wu).dot(self.Q.T).dot(Cu).dot(self.phi[u, :])
                
            loss = (self.C*(self.phi - self.P.dot(self.Q.T))**2).sum()
            l2 = (self.P ** 2).sum() + (self.Q ** 2).sum()
            self.loss.append(loss + self.lambd * l2)
            

    def predict(self, u, i):
        return self.P[u].dot(self.Q[i])

In [5]:
np.random.seed(42)

In [6]:
stars = np.arange(6)
p = np.array([10, 1, 1, 1, 1, 1])
m = 5
n = 10

ratings = np.random.choice(stars, size=m*n, p=p / p.sum()).reshape((m, n))

In [7]:
ratings

array([[0, 5, 1, 0, 0, 0, 0, 3, 0, 1],
       [0, 5, 3, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 2, 0, 0, 0, 0],
       [0, 0, 0, 5, 5, 3, 0, 0, 1, 0],
       [0, 0, 0, 4, 0, 0, 0, 0, 0, 0]])

In [8]:
ratings.shape

(5, 10)

In [18]:
als = ImplicitALS(R=ratings, k=20)

In [19]:
als.train()

In [20]:
als.loss

[211.94569147839772,
 169.06820382523924,
 152.8085030072954,
 143.32343774545967,
 137.29155772090473,
 133.25237275557507,
 130.44741071099253,
 128.44872519455436,
 126.99819381403191,
 125.93138903469803]

In [22]:
als.Q.shape

(10, 20)

In [16]:
als.P

array([[-0.02045395,  0.54534051,  0.9803006 ],
       [ 0.14683239,  0.79106706,  0.58534218],
       [ 0.39765444, -0.58765716,  0.53592615],
       [ 1.01651946, -0.46383274,  0.32479014],
       [ 0.89263978,  0.05379664, -0.17020861]])

In [17]:
als.P[-1, :].dot(als.Q.T)

array([ 0.        ,  0.05509913,  0.05033573,  0.94034237,  0.5895627 ,
        0.32389258,  0.        , -0.1207522 ,  0.51100916, -0.1067449 ])